In [1]:
from datasets import load_dataset
from transformers import AutoModel
import torch
from torchvision import transforms
import PIL

In [2]:
model = AutoModel.from_pretrained('facebook/dinov2-giant')
model = model.to("cuda")

In [3]:
vimeo6k = load_dataset("danjacobellis/vimeo90k_triplet", split="train[:6000]")

Resolving data files:   0%|          | 0/71 [00:00<?, ?it/s]

In [4]:
def extract_dinov2_features(sample):
    img = sample['image'].crop((0,2,448,254))
    with torch.no_grad():
        x = transforms.ToTensor()(img).unsqueeze(0).to("cuda")
        y = model.forward(x)[0]
        sample['cls_token'] = y[:, 0].detach().cpu()
        sample['patch_tokens'] = y[:, 1:].detach().cpu().reshape((1,18,32,1536)).permute((0,3,1,2))
        return sample

In [5]:
vimeo6k_dino = vimeo6k.map(extract_dinov2_features)
vimeo6k_dino = vimeo6k_dino.remove_columns("image")

In [ ]:
vimeo6k_dino.push_to_hub("danjacobellis/vimeo6k_dino",split="train")

Pushing dataset shards to the dataset hub:   0%|          | 0/44 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
download = load_dataset("danjacobellis/vimeo6k_dino", split="train")